#### Import Modules

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
###### import os
import pandas as pd
from glob import glob 
from dotenv import load_dotenv
environment_loaded = load_dotenv()

#### Instantiate dask optimisations

In [ ]:
d = {
    'GPS': 
        [{'_id': 1, 'action': 'Long', 'units': 20, 'date': '2022-02-03', 'price': 14.57, 'simulatedFee': 0}
        ,{'_id': 2, 'action': 'Short', 'units': 20, 'date': '2022-04-04', 'price': 14.21, 'simulatedFee': 0}
        ,{'_id': 3, 'action': 'Long', 'units': 25, 'date': '2022-06-13', 'price': 9.03, 'simulatedFee': 0.2}],
    'IAU': 
        [{'_id': 1, 'action': 'Long', 'units': 10, 'date': '2022-03-02', 'price': 36.61, 'simulatedFee': 0}
        ,{'_id': 2, 'action': 'Short', 'units': 10, 'date': '2022-03-08', 'price': 38.98, 'simulatedFee': 0}
        ,{'_id': 3, 'action': 'Long', 'units': 5, 'date': '2022-03-15', 'price': 36.45, 'simulatedFee': 0.1}
        ,{'_id': 4, 'action': 'Long', 'units': 5, 'date': '2022-04-13', 'price': 37.62, 'simulatedFee': 0.23}],
    'M': 
        [{'_id': 1, 'action': 'Long', 'units': 55, 'date': '2022-05-24', 'price': 17.61, 'simulatedFee': 0}],
    'NFLX': 
        [{'_id': 1, 'action': 'Long', 'units': 2, 'date': '2022-01-19', 'price': 515.86, 'simulatedFee': 0}],
    'STNG': 
        [{'_id': 1, 'action': 'Long', 'units': 50, 'date': '2022-04-24', 'price': 20.60, 'simulatedFee': 0.26}
        ,{'_id': 2, 'action': 'Long', 'units': 20, 'date': '2022-05-13', 'price': 28.35, 'simulatedFee': 0.29}
        ,{'_id': 3, 'action': 'Short', 'units': 45, 'date': '2022-05-24', 'price': 30.46, 'simulatedFee': 0.31}
        ,{'_id': 4, 'action': 'Long', 'units': 20, 'date': '2022-06-02', 'price': 35.19, 'simulatedFee': 0.35}]
}

#### Initiate mulithreading 

In [228]:
from typing import Callable, Dict, Any
from api.gcp.connect_cloud import CloudUtility
from concurrent.futures import ThreadPoolExecutor

In [229]:
cloud_util = CloudUtility()

In [262]:
{}

dict

In [263]:
isinstance([], list)

True

In [230]:
def parallelize_factory(
    f: Callable, 
    iterables_params: Dict[str, Any],
    constants_params: Dict[str, Any] = None):
    
    data = []
    assert_accumulator = None
    
    
    if not isinstance(iterables_params, dict):
        raise ValueError(f"Please ensure that iterables_params are of type Dict[str, Any]. You entered type {type(iterables_params)}.")
        
    if not isinstance(constants_params, dict) and constants_params is not None:
        raise ValueError(f"Please ensure that constants_params are of type Dict[str, Any]. You entered type {type(constants_params)}.")
    
    for key, value in iterables_params.items(): 
        
        if assert_accumulator is not None: 
            if len(value) == assert_accumulator:
                raise ValueError(f"Please ensure values in list are of similar length.")
        
        if not isinstance(value, list):
            raise ValueError(f"Please ensure that values of iterables params are of type {list.__name__}. You passed a value of type {type(value)}")
            
        assert_accumulator = value
        
    if constants_params is not None:
        worker_job = partial(f, constant_params = constants_params)
    else: 
        worker_job = f
    
    with ThreadPoolExecutor(max_workers = min(12, len(iterables_params))) as executor:

        futures = []
        
        for worker_api_key_list, worker_parameter_list in zip(worker_api_keys, worker_parameters):
            
            job = executor.submit(
                f,
                **iterables_params
            )

            futures.append(job)

    for future in as_completed(futures):
        data.append(future.result())

    return list(itertools.chain.from_iterable(data))

In [241]:
parallelize_factory(cloud_util.get_file_with_prefix_from_gcs, 
    {"prefix": ["semantic-ion/tickers/historical_ticks_5/AAPL_", "semantic-ion/tickers/historical_ticks_5/GPS_"]})

['semantic-ion/tickers/historical_ticks_5/AAPL_', 'semantic-ion/tickers/historical_ticks_5/GPS_']


[]

### Static Analysis

#### Instantiate sample data

In [ ]:
s = {
    'GPS': 
        [{'_id': 1, 'action': 'Long', 'units': 20, 'date': '', 'price': 14.57, 'simulatedFee': 0}],
    'IAU': 
        [{'_id': 1, 'action': 'Long', 'units': 10, 'date': '', 'price': 36.61, 'simulatedFee': 0}],
    'M': 
        [{'_id': 1, 'action': 'Long', 'units': 55, 'date': '', 'price': 17.61, 'simulatedFee': 0}],
    'NFLX': 
        [{'_id': 1, 'action': 'Long', 'units': 2, 'date': '', 'price': 515.86, 'simulatedFee': 0}],
    'STNG': 
        [{'_id': 1, 'action': 'Long', 'units': 50, 'date': '', 'price': 20.60, 'simulatedFee': 0.26}]
}

In [243]:
data = {
    'GPS': np.random.random(9600) * 10 + 50,
    'NFLX': np.random.random(9600) * 10 + 200,
    'M': np.random.random(9600) * 10 + 10,
    'IAU': np.random.random(9600) * 10 + 15,
    'STNG': np.random.random(9600) * 10 + 20,
}

weights = np.array([0.2, 0.1, 0.2, 0.3, 0.2])

In [135]:
ts_arr = np.array([*data.values()])
pf_raw_value_ot = ts_arr.T.dot(weights)

In [165]:
def pad_array(arr: np.ndarray, size: tuple):
    """ Pad a 1D numpy array to match size requirements
    """
    repad_size = size - arr.size
    return np.pad(arr, (repad_size, ), mode = 'constant', constant_values = (np.nan,))[:-1]

In [167]:
def calculate_returns(
    arr: np.ndarray,
    ctype: str = "log"):
    """ Calculate the returns given prices
    Params
    ----------
    arr       : A one dimensional numpy array containing the closing prices of the asset 
    ctype     : The return calculation type. Available selections are: log, 
    """
    
    assert arr.ndim == 1, "Pass in only one dimensional np arrays"
    
    if ctype == "log":
        return pad_array(np.log(1 + np.diff(arr)/arr[1:]), arr.size)
    else: 
        raise ValueError("Please enter a valid ctype: log")

### Dynamic Portfolio Metrics

In [ ]:
from typing import Iterable

def days_open(date_series: Iterable) -> np.uint16:
    return np.uint16((date_series.max() - date_series.min()).days)

def turnover_rate(date_series: Iterable) -> np.float64:
    date_series = np.fromiter(date_series, dtype='datetime64[D]')
    days_traded = days_open(date_series)
    return np.round(date_series.size / days_traded, 4)